In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [3]:
MME = pd.read_csv('MME_.csv', encoding = 'euc-kr', low_memory = False)
MME

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13400.0,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15700.0,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14700.0,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19500.0,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,1.641,60.9,74.0,88.0,NaN,...,NaN,NaN,17900.0,NaN,0,92.0,62.0,74.201919,22.615184,1
70206,A07,201407.0,NIH1604999732,75.0,1,1.632,50.8,81.3,110.0,NaN,...,NaN,NaN,10500.0,NaN,0,131.0,66.0,68.312024,19.073193,3
70207,A07,201409.0,NIH1604999772,55.0,1,1.635,66.2,88.3,85.0,NaN,...,NaN,NaN,13900.0,NaN,0,NaN,NaN,50.048887,24.764096,2
70208,A07,201403.0,NIH1604999929,53.0,2,1.530,63.2,85.3,95.0,NaN,...,NaN,NaN,27100.0,NaN,0,112.0,71.0,74.230720,26.998163,2


In [10]:
weight_cycle = MME[['기수', 'NIHID', 'EDATE', 'WEIGHT']]
weight_cycle

,기수,NIHID,EDATE,WEIGHT
0,A01,NIH1604000095,200209.0,48.0
1,A01,NIH1604000171,200201.0,75.0
2,A01,NIH1604000338,200210.0,50.4
3,A01,NIH1604000362,200201.0,53.6
4,A01,NIH1604000424,200205.0,88.0
...,...,...,...,...
70205,A07,NIH1604999643,201303.0,60.9
70206,A07,NIH1604999732,201407.0,50.8
70207,A07,NIH1604999772,201409.0,66.2
70208,A07,NIH1604999929,201403.0,63.2


In [11]:
weight_cycle = weight_cycle.loc[(weight_cycle['기수'] >= 'A01') & (weight_cycle['기수'] <= 'A05')]
weight_cycle

,기수,NIHID,EDATE,WEIGHT
0,A01,NIH1604000095,200209.0,48.0
1,A01,NIH1604000171,200201.0,75.0
2,A01,NIH1604000338,200210.0,50.4
3,A01,NIH1604000362,200201.0,53.6
4,A01,NIH1604000424,200205.0,88.0
...,...,...,...,...
50145,A05,NIH1604999643,200909.0,60.1
50146,A05,NIH1604999732,201003.0,57.6
50147,A05,NIH1604999772,NaN,NaN
50148,A05,NIH1604999929,NaN,NaN


In [12]:
weight_cycle = weight_cycle.dropna().reset_index(drop = True)
weight_cycle

,기수,NIHID,EDATE,WEIGHT
0,A01,NIH1604000095,200209.0,48.0
1,A01,NIH1604000171,200201.0,75.0
2,A01,NIH1604000338,200210.0,50.4
3,A01,NIH1604000362,200201.0,53.6
4,A01,NIH1604000424,200205.0,88.0
...,...,...,...,...
39222,A05,NIH1604999204,200911.0,57.3
39223,A05,NIH1604999446,200911.0,77.5
39224,A05,NIH1604999640,200908.0,55.3
39225,A05,NIH1604999643,200909.0,60.1


In [14]:
unique_id = weight_cycle['NIHID'].unique().tolist()

In [29]:
# 5기의 데이터 5 * 4개가 모두 있는 사람 뽑아내기
finalDf = pd.DataFrame()
for id in unique_id: 
    df = pd.DataFrame(weight_cycle.loc[weight_cycle['NIHID'] == id])
    if len(df) == 5: # 해당 id에 대해서 전체 5개의 기수 데이터가 다 있다면
        df['lag_WEIGHT'] = df['WEIGHT'].shift(1) # lag 생성
        df['lag2_WEIGHT'] = df['WEIGHT'].shift(2) # lag 생성
        df['lag3_WEIGHT'] = df['WEIGHT'].shift(3) # lag 생성
        df['lag4_WEIGHT'] = df['WEIGHT'].shift(4) # lag 생성
        finalDf = pd.concat([finalDf, df], axis = 0)

In [30]:
finalDf.reset_index(inplace = True, drop = True)

,기수,NIHID,EDATE,WEIGHT,lag_WEIGHT,lag2_WEIGHT,lag3_WEIGHT,lag4_WEIGHT
0,A01,NIH1604000171,200201.0,75.0,NaN,NaN,NaN,NaN
1,A02,NIH1604000171,200311.0,75.6,75.0,NaN,NaN,NaN
2,A03,NIH1604000171,200510.0,75.5,75.6,75.0,NaN,NaN
3,A04,NIH1604000171,200712.0,75.5,75.5,75.6,75.0,NaN
4,A05,NIH1604000171,200911.0,79.8,75.5,75.5,75.6,75.0
...,...,...,...,...,...,...,...,...
27310,A01,NIH1604999643,200109.0,61.0,NaN,NaN,NaN,NaN
27311,A02,NIH1604999643,200305.0,67.5,61.0,NaN,NaN,NaN
27312,A03,NIH1604999643,200601.0,62.2,67.5,61.0,NaN,NaN
27313,A04,NIH1604999643,200711.0,60.3,62.2,67.5,61.0,NaN


In [31]:
finalDf.head(30)

,기수,NIHID,EDATE,WEIGHT,lag_WEIGHT,lag2_WEIGHT,lag3_WEIGHT,lag4_WEIGHT
0,A01,NIH1604000171,200201.0,75.0,NaN,NaN,NaN,NaN
1,A02,NIH1604000171,200311.0,75.6,75.0,NaN,NaN,NaN
2,A03,NIH1604000171,200510.0,75.5,75.6,75.0,NaN,NaN
3,A04,NIH1604000171,200712.0,75.5,75.5,75.6,75.0,NaN
4,A05,NIH1604000171,200911.0,79.8,75.5,75.5,75.6,75.0
5,A01,NIH1604000338,200210.0,50.4,NaN,NaN,NaN,NaN
6,A02,NIH1604000338,200408.0,52.7,50.4,NaN,NaN,NaN
7,A03,NIH1604000338,200608.0,53.1,52.7,50.4,NaN,NaN
8,A04,NIH1604000338,200811.0,54.7,53.1,52.7,50.4,NaN
9,A05,NIH1604000338,201008.0,53.3,54.7,53.1,52.7,50.4


In [18]:
len(finalDf['NIHID'].unique())

5463

,기수,NIHID,EDATE,WEIGHT,lag_WEIGHT
0,A01,NIH1604000171,200201.0,75.0,75.6
1,A02,NIH1604000171,200311.0,75.6,75.5
2,A03,NIH1604000171,200510.0,75.5,75.5
3,A04,NIH1604000171,200712.0,75.5,79.8
4,A05,NIH1604000171,200911.0,79.8,50.4
...,...,...,...,...,...
27310,A01,NIH1604999643,200109.0,61.0,67.5
27311,A02,NIH1604999643,200305.0,67.5,62.2
27312,A03,NIH1604999643,200601.0,62.2,60.3
27313,A04,NIH1604999643,200711.0,60.3,60.1
